In [4]:
import blpapi
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import itertools
import os
import math
import json
import multiprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

# Parameters
class FuturesParams:
    MIN_VOLUME = 0
    MIN_DAYS_TO_EXPIRY = 0
    START_YEAR = 1985
    END_YEAR = 2026
    MAX_MONTHS_FORWARD = 13
    TRADING_DAYS_PER_YEAR = 251
    COMMODITIES = ['CLA', 'COA', 'XBA', 'HOA', 'NGA', 'FNA', 'HG', 'ALE', 'GC', 'SI']

# Bloomberg Server API settings
HOST = 'localhost'
PORT = 8194

def setup_commodity_folders(commodity):
    """Create and return paths for a commodity's data"""
    base_dir = os.getcwd()  # This should be your git repository root
    data_dir = os.path.join(base_dir, 'data', commodity)
    viz_dir = os.path.join(base_dir, 'visualizations')
    excel_dir = os.path.join(base_dir, 'excel')
    
    # Create directories if they don't exist
    for directory in [data_dir, viz_dir, excel_dir]:
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(f"Created directory: {directory}")
    
    return {
        'data': data_dir,
        'visualizations': viz_dir,
        'excel': excel_dir
    }

def start_bloomberg_session():
    """Initialize Bloomberg API session"""
    session_options = blpapi.SessionOptions()
    session_options.setServerHost(HOST)
    session_options.setServerPort(PORT)
    session = blpapi.Session(session_options)
    if not session.start():
        raise Exception("Failed to start session.")
    if not session.openService("//blp/refdata"):
        raise Exception("Failed to open service")
    return session

def generate_futures_tickers(commodity):
    """Generate all possible futures tickers for a commodity."""
    months = 'FGHJKMNQUVXZ'  # Bloomberg month codes
    tickers = []
    for year in range(FuturesParams.START_YEAR, FuturesParams.END_YEAR + 1):
        year_str = str(year)[-2:]  # Get last 2 digits
        for month in months:
            tickers.append(f"{commodity}{month}{year_str} Comdty")
    print(f"Generated {len(tickers)} tickers for {commodity}")
    return tickers

def fetch_bloomberg_data_batch(session, securities_batch, fields, start_date, end_date):
    """Fetch historical data for a batch of securities"""
    refDataService = session.getService("//blp/refdata")
    
    request = refDataService.createRequest("HistoricalDataRequest")
    for security in securities_batch:
        request.append("securities", security)
    for field in fields:
        request.append("fields", field)
    request.set("startDate", start_date.strftime("%Y%m%d"))
    request.set("endDate", end_date.strftime("%Y%m%d"))
    request.set("periodicityAdjustment", "ACTUAL")
    request.set("periodicitySelection", "DAILY")
    
    data_dict = {}
    
    try:
        session.sendRequest(request)
        
        while True:
            event = session.nextEvent()
            if event.eventType() in (blpapi.Event.PARTIAL_RESPONSE, blpapi.Event.RESPONSE):
                for msg in event:
                    securityData = msg.getElement("securityData")
                    security_name = securityData.getElementAsString("security")
                    fieldDataArray = securityData.getElement("fieldData")
                    
                    for i in range(fieldDataArray.numValues()):
                        fieldData = fieldDataArray.getValueAsElement(i)
                        date = fieldData.getElementAsDatetime("date")
                        
                        if date not in data_dict:
                            data_dict[date] = {}
                        
                        current_data = {}
                        for field in fields:
                            if fieldData.hasElement(field):
                                current_data[f"{security_name}_{field}"] = fieldData.getElementAsFloat(field)
                                
                        data_dict[date].update(current_data)
                                
            if event.eventType() == blpapi.Event.RESPONSE:
                break
                
    except Exception as e:
        print(f"Warning: Error fetching data for batch: {e}")
        return pd.DataFrame()
    
    if data_dict:
        df = pd.DataFrame.from_dict(data_dict, orient='index')
        df.index = pd.to_datetime(df.index)
        df.sort_index(inplace=True)
        return df
    return pd.DataFrame()

def fetch_bloomberg_metadata_batch(session, securities_batch):
    """Fetch metadata for a batch of securities"""
    refDataService = session.getService("//blp/refdata")
    metadata = {}
    
    try:
        request = refDataService.createRequest("ReferenceDataRequest")
        for security in securities_batch:
            request.append("securities", security)
        request.append("fields", "name")
        request.append("fields", "QUOTE_UNITS")
        request.append("fields", "LAST_TRADEABLE_DT")
        
        session.sendRequest(request)
        
        while True:
            event = session.nextEvent()
            if event.eventType() in (blpapi.Event.PARTIAL_RESPONSE, blpapi.Event.RESPONSE):
                for msg in event:
                    securityDataArray = msg.getElement("securityData")
                    for i in range(securityDataArray.numValues()):
                        securityData = securityDataArray.getValueAsElement(i)
                        ticker = securityData.getElementAsString("security")
                        fieldData = securityData.getElement("fieldData")
                        
                        metadata[ticker] = {
                            'name': fieldData.getElementAsString("name") if fieldData.hasElement("name") else ticker,
                            'units': fieldData.getElementAsString("QUOTE_UNITS") if fieldData.hasElement("QUOTE_UNITS") else '',
                            'last_trade_date': fieldData.getElementAsString("LAST_TRADEABLE_DT") if fieldData.hasElement("LAST_TRADEABLE_DT") else ''
                        }
            if event.eventType() == blpapi.Event.RESPONSE:
                break
                
    except Exception as e:
        print(f"Warning: Error fetching metadata for batch: {e}")
    
    return metadata

def fetch_interest_rates(session, start_date, end_date):
    """Fetch daily interest rates (GB12 Govt yields)"""
    print("\nFetching interest rate data...")
    try:
        rate_data = fetch_bloomberg_data_batch(
            session, 
            ["GB12 Govt"], 
            ["PX_LAST"], 
            start_date, 
            end_date
        )
        
        if not rate_data.empty:
            # Clean up the column name
            rate_data.columns = ['rate']
            # Convert to percentage
            rate_data = rate_data / 100
            print("Successfully fetched interest rate data")
            return rate_data
        
    except Exception as e:
        print(f"Error fetching interest rate data: {e}")
    
    print("Warning: Using zero interest rates due to data fetch failure")
    return pd.DataFrame(0, index=pd.date_range(start_date, end_date), columns=['rate'])

def process_price_volume_data(df):
    """Process raw data to create price and volume dataframes"""
    try:
        print(f"Processing raw data with {len(df.columns)} columns")
        
        price_cols = [col for col in df.columns if 'PX_LAST' in col]
        volume_cols = [col for col in df.columns if 'PX_VOLUME' in col]
        
        print(f"Found {len(price_cols)} price columns and {len(volume_cols)} volume columns")
        
        # Create separate dataframes for prices and volumes
        prices_df = df[price_cols].copy()
        volumes_df = df[volume_cols].copy()
        
        # Clean up column names
        prices_df.columns = [col.replace('_PX_LAST', '') for col in prices_df.columns]
        volumes_df.columns = [col.replace('_PX_VOLUME', '') for col in volumes_df.columns]
        
        # Set price to NaN where volume is below threshold
        for contract in prices_df.columns:
            if contract in volumes_df.columns:
                mask = (volumes_df[contract].isna()) | (volumes_df[contract] < FuturesParams.MIN_VOLUME)
                prices_df.loc[mask, contract] = np.nan
        
        # Sort columns chronologically
        prices_df = prices_df.reindex(sorted(prices_df.columns), axis=1)
        volumes_df = volumes_df.reindex(sorted(volumes_df.columns), axis=1)
        
        # Remove columns with all NaN values
        prices_df = prices_df.dropna(axis=1, how='all')
        volumes_df = volumes_df.dropna(axis=1, how='all')
        
        print(f"Processed {len(prices_df.columns)} active futures")
        return prices_df, volumes_df
        
    except Exception as e:
        print(f"Error in process_price_volume_data: {str(e)}")
        raise

def get_last_trade_dates(prices_df, metadata_dict):
    """Create mapping of contracts to their last trade dates"""
    last_trade_dates = {}
    
    for contract in prices_df.columns:
        # Try to get date from metadata first
        meta_date = None
        if contract in metadata_dict:
            try:
                meta_date = pd.to_datetime(metadata_dict[contract]['last_trade_date'])
            except:
                pass
        
        # If no metadata date, use last price date
        if meta_date is None:
            meta_date = prices_df[contract].last_valid_index()
        
        if meta_date is not None:
            last_trade_dates[contract] = meta_date
    
    print(f"Determined last trade dates for {len(last_trade_dates)} contracts")
    return last_trade_dates

def calculate_days_to_expiry(date, last_trade_dates):
    """Calculate days to expiry for each contract from a given date"""
    days_to_expiry = {}
    for contract, last_trade_date in last_trade_dates.items():
        if last_trade_date >= date:
            days = (last_trade_date - date).days
            if days >= FuturesParams.MIN_DAYS_TO_EXPIRY:
                days_to_expiry[contract] = days
    return days_to_expiry

def identify_roll_dates(monthly_futures_df):
    """Identify dates when the front month future changes"""
    roll_dates = []
    previous_front = None
    front_month_col = 'month_1_future'
    
    if front_month_col not in monthly_futures_df.columns:
        return roll_dates
        
    for date in monthly_futures_df.index:
        current_front = monthly_futures_df.loc[date, front_month_col]
        if pd.notna(current_front) and current_front != previous_front:
            roll_dates.append(date)
            previous_front = current_front
    
    print(f"Identified {len(roll_dates)} roll dates")
    return roll_dates

def create_monthly_futures_data(prices_df, metadata_dict, commodity, interest_rates_df=None):
    """Create dataframe with futures ordered by month and calculate spreads"""
    print(f"\nOrganizing futures data for {commodity}...")
    
    # Get last trade dates for all contracts
    last_trade_dates = get_last_trade_dates(prices_df, metadata_dict)
    
    # Initialize dataframes for results
    monthly_futures = pd.DataFrame(index=prices_df.index)
    spreads_dollar = pd.DataFrame(index=prices_df.index)
    spreads_percent = pd.DataFrame(index=prices_df.index)
    spreads_percent_annual = pd.DataFrame(index=prices_df.index)
    spreads_percent_annual_adjusted = pd.DataFrame(index=prices_df.index)
    days_to_expiry_df = pd.DataFrame(index=prices_df.index)
    
    print(f"Processing {len(prices_df.index)} dates...")
    processed_dates = 0
    
    for date in prices_df.index:
        # Get valid contracts for this date
        valid_contracts = [c for c in prices_df.columns if not pd.isna(prices_df.loc[date, c])]
        if not valid_contracts:
            continue
        
        # Calculate days to expiry and filter contracts
        days_to_expiry = calculate_days_to_expiry(date, last_trade_dates)
        valid_contracts = [c for c in valid_contracts if c in days_to_expiry]
        
        if not valid_contracts:
            continue
        
        # Order contracts by expiry
        ordered_contracts = sorted(valid_contracts, 
                                 key=lambda x: days_to_expiry.get(x, float('inf')))
        
        # Store prices and days to expiry for each month
        for i, contract in enumerate(ordered_contracts, 1):
            if i > FuturesParams.MAX_MONTHS_FORWARD:
                break
            monthly_futures.loc[date, f"month_{i}_future"] = contract
            monthly_futures.loc[date, f"month_{i}_price"] = prices_df.loc[date, contract]
            days_to_expiry_df.loc[date, f"month_{i}_days_to_expiry"] = days_to_expiry.get(contract)
        
        # Calculate spreads if we have at least two months
        if len(ordered_contracts) >= 2:
            m1_contract = ordered_contracts[0]
            m1_price = prices_df.loc[date, m1_contract]
            m1_days = days_to_expiry.get(m1_contract)
            
            # Calculate spreads up to max months forward
            for i in range(1, min(FuturesParams.MAX_MONTHS_FORWARD-1, len(ordered_contracts))):
                far_contract = ordered_contracts[i]
                far_price = prices_df.loc[date, far_contract]
                far_days = days_to_expiry.get(far_contract)
                
                if m1_price != 0 and far_days and m1_days:
                    # Dollar spread
                    dollar_spread = m1_price - far_price
                    spreads_dollar.loc[date, f"spread_1_{i+1}m"] = dollar_spread
                    
                    # Basic percentage spread
                    pct_spread = dollar_spread / m1_price
                    spreads_percent.loc[date, f"spread_1_{i+1}m_pct"] = pct_spread
                    
                    # Calculate days difference for annualization
                    days_difference = far_days - m1_days
                    if days_difference > 0:
                        # Annualize the spread
                        annual_factor = FuturesParams.TRADING_DAYS_PER_YEAR / days_difference
                        annual_spread = pct_spread * annual_factor
                        spreads_percent_annual.loc[date, f"spread_1_{i+1}m_pct_annual"] = annual_spread
                        
                        # Add interest rate adjustment
                        if interest_rates_df is not None and date in interest_rates_df.index:
                            rate = interest_rates_df.loc[date, 'rate']
                            adjusted_spread = annual_spread + rate  # Adding interest rate
                            spreads_percent_annual_adjusted.loc[date, f"spread_1_{i+1}m_pct_annual_adj"] = adjusted_spread
        
        processed_dates += 1
        if processed_dates % 100 == 0:
            print(f"Processed {processed_dates} dates...")
    
    print(f"Completed futures data organization. Processed {processed_dates} dates with valid data.")
    return (monthly_futures, spreads_dollar, spreads_percent, 
            spreads_percent_annual, spreads_percent_annual_adjusted, days_to_expiry_df)

def process_commodity_data(session, commodity, start_date, end_date, metadata_batch_size=200, price_batch_size=50):
    """Process data for a single commodity"""
    # Generate tickers for this commodity
    tickers = generate_futures_tickers(commodity)
    
    # First fetch all metadata
    print(f"\nProcessing metadata for {commodity}...")
    metadata_batches = np.array_split(tickers, math.ceil(len(tickers)/metadata_batch_size))
    all_metadata = {}
    
    for i, batch in enumerate(metadata_batches, 1):
        print(f"Processing metadata batch {i}/{len(metadata_batches)} for {commodity}")
        try:
            batch_metadata = fetch_bloomberg_metadata_batch(session, batch)
            all_metadata.update(batch_metadata)
        except Exception as e:
            print(f"Warning: Error processing metadata batch {i} for {commodity}: {e}")
            continue
    
    print(f"Collected metadata for {len(all_metadata)} contracts")
    
    # Then fetch price and volume data
    print(f"\nProcessing price and volume data for {commodity}...")
    price_batches = np.array_split(tickers, math.ceil(len(tickers)/price_batch_size))
    all_data = pd.DataFrame()
    
    for i, batch in enumerate(price_batches, 1):
        print(f"Processing batch {i}/{len(price_batches)} for {commodity}")
        try:
            batch_data = fetch_bloomberg_data_batch(session, batch, ["PX_LAST", "PX_VOLUME"], 
                                                  start_date, end_date)
            if not batch_data.empty:
                if all_data.empty:
                    all_data = batch_data
                else:
                    new_cols = [col for col in batch_data.columns if col not in all_data.columns]
                    if new_cols:
                        batch_data = batch_data[new_cols]
                        all_data = pd.concat([all_data, batch_data], axis=1)
        except Exception as e:
            print(f"Warning: Error processing batch {i}: {str(e)}")
            continue
    
    # Process price and volume data
    if not all_data.empty:
        print(f"Processing combined data with {len(all_data.columns)} columns")
        try:
            prices_df, volumes_df = process_price_volume_data(all_data)
            
            if not prices_df.empty:
                print(f"Successfully processed {len(prices_df.columns)} active futures")
                return prices_df, volumes_df, all_metadata
            else:
                print("No valid price data found after processing")
                return pd.DataFrame(), pd.DataFrame(), all_metadata
                
        except Exception as e:
            print(f"Error processing data: {str(e)}")
            return pd.DataFrame(), pd.DataFrame(), all_metadata
    else:
        print("No data received from Bloomberg")
        return pd.DataFrame(), pd.DataFrame(), all_metadata


def create_spread_visualizations(spreads_dollar, spreads_percent, spreads_percent_annual, 
                               spreads_percent_annual_adjusted, monthly_futures, commodity, paths):
    """Create visualizations for all types of spreads in a single PDF"""
    print(f"\nCreating visualizations for {commodity}...")
    
    # Identify roll dates
    roll_dates = identify_roll_dates(monthly_futures)
    print(f"Plotting with {len(roll_dates)} roll dates")
    
    def create_spread_plot(df, title, ylabel, percentage=False):
        plt.figure(figsize=(15, 8))
        
        # Plot spreads
        for column in df.columns:
            plt.plot(df.index, df[column], label=column, linewidth=1.5)
        
        # Add roll date lines
        for roll_date in roll_dates:
            plt.axvline(x=roll_date, color='gray', linewidth=0.5, alpha=0.7)
        
        plt.title(title)
        plt.xlabel('Date')
        plt.ylabel(ylabel)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        
        if percentage:
            plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.1%}'.format(y)))
        
        plt.grid(True, axis='y')
        plt.tight_layout()
    
    # Create PDF with all visualizations
    pdf_path = os.path.join(paths['visualizations'], f'{commodity}_spreads.pdf')
    with PdfPages(pdf_path) as pdf:
        # Dollar spreads
        create_spread_plot(
            spreads_dollar,
            f'{commodity} Dollar Spreads',
            'Spread Value'
        )
        pdf.savefig()
        plt.close()
        
        # Percentage spreads
        create_spread_plot(
            spreads_percent,
            f'{commodity} Percentage Spreads',
            'Spread Percentage',
            percentage=True
        )
        pdf.savefig()
        plt.close()
        
        # Annualized percentage spreads
        create_spread_plot(
            spreads_percent_annual,
            f'{commodity} Annualized Percentage Spreads',
            'Annualized Spread Percentage',
            percentage=True
        )
        pdf.savefig()
        plt.close()
        
        # Interest rate adjusted annualized spreads
        create_spread_plot(
            spreads_percent_annual_adjusted,
            f'{commodity} Interest Rate Adjusted Annualized Spreads',
            'Adjusted Annualized Spread Percentage',
            percentage=True
        )
        pdf.savefig()
        plt.close()
    
    print(f"Visualizations saved to {pdf_path}")

def export_for_observable(spreads_dollar, spreads_percent, spreads_percent_annual, 
                        spreads_percent_annual_adjusted, monthly_futures, commodity, paths):
    """Export data in a format suitable for Observable"""
    
    def format_for_export(df, prefix):
        # Reset index to make date a column
        df_export = df.reset_index()
        # Rename date column
        df_export = df_export.rename(columns={'index': 'date'})
        # Convert date to string in ISO format
        df_export['date'] = df_export['date'].dt.strftime('%Y-%m-%d')
        # Create JSON-friendly column names
        df_export.columns = [col.replace(' ', '_').lower() for col in df_export.columns]
        return df_export

    # Format each type of spread
    dollar_export = format_for_export(spreads_dollar, 'dollar')
    percent_export = format_for_export(spreads_percent, 'percent')
    annual_export = format_for_export(spreads_percent_annual, 'annual')
    adjusted_export = format_for_export(spreads_percent_annual_adjusted, 'adjusted')

    # Get roll dates
    roll_dates = identify_roll_dates(monthly_futures)
    roll_dates_list = [d.strftime('%Y-%m-%d') for d in roll_dates]

    # Save as JSON files
    output_files = {}
    try:
        for name, df in [
            ('dollar', dollar_export),
            ('percent', percent_export),
            ('annual', annual_export),
            ('adjusted', adjusted_export)
        ]:
            filename = os.path.join(paths['data'], f'{commodity}_{name}_spreads.json')
            df.to_json(filename, orient='records', date_format='iso')
            output_files[name] = filename
            print(f"Exported {name} spreads to {filename}")

        # Save metadata with roll dates
        metadata = {
            'commodity': commodity,
            'date_range': {
                'start': str(spreads_dollar.index.min().date()),
                'end': str(spreads_dollar.index.max().date())
            },
            'series_info': {
                'dollar': list(spreads_dollar.columns),
                'percent': list(spreads_percent.columns),
                'annual': list(spreads_percent_annual.columns),
                'adjusted': list(spreads_percent_annual_adjusted.columns)
            },
            'roll_dates': roll_dates_list
        }
        
        metadata_file = os.path.join(paths['data'], f'{commodity}_metadata.json')
        with open(metadata_file, 'w') as f:
            json.dump(metadata, f, indent=2)
        output_files['metadata'] = metadata_file
        print(f"Exported metadata to {metadata_file}")
        
    except Exception as e:
        print(f"Error exporting data: {e}")
        
    return output_files

def process_single_commodity(commodity, start_date, end_date):
    """Process a single commodity - to be run in parallel"""
    try:
        print(f"\nStarting processing for {commodity}")
        session = start_bloomberg_session()
        
        try:
            # Setup paths for this commodity
            paths = setup_commodity_folders(commodity)
            
            # Get raw data
            prices_df, volumes_df, metadata = process_commodity_data(
                session, commodity, start_date, end_date, 
                metadata_batch_size=200, price_batch_size=50
            )
            
            if prices_df.empty:
                print(f"No price data found for {commodity}")
                return None
            
            # Filter metadata to only include active futures
            active_futures = prices_df.columns
            metadata = {k: v for k, v in metadata.items() if k in active_futures}
            
            # Get last trade dates
            last_trade_dates = get_last_trade_dates(prices_df, metadata)
            
            # Get interest rates
            interest_rates_df = fetch_interest_rates(session, start_date, end_date)
            
            # Create monthly futures data and calculate spreads
            (monthly_futures, spreads_dollar, spreads_percent, 
             spreads_percent_annual, spreads_percent_annual_adjusted, 
             days_to_expiry) = create_monthly_futures_data(
                prices_df, metadata, commodity, interest_rates_df)
            
            # Export data for Observable
            output_files = export_for_observable(
                spreads_dollar, spreads_percent, spreads_percent_annual,
                spreads_percent_annual_adjusted, monthly_futures, 
                commodity, paths
            )
            
            # Create visualizations
            create_spread_visualizations(
                spreads_dollar, spreads_percent, spreads_percent_annual,
                spreads_percent_annual_adjusted, monthly_futures,
                commodity, paths
            )
            
            # Save Excel file
            excel_path = os.path.join(paths['excel'], f'futures_analysis_{commodity}.xlsx')
            with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
                prices_df.to_excel(writer, sheet_name='Prices')
                volumes_df.to_excel(writer, sheet_name='Volumes')
                monthly_futures.to_excel(writer, sheet_name='Monthly')
                days_to_expiry.to_excel(writer, sheet_name='DaysToExpiry')
                spreads_dollar.to_excel(writer, sheet_name='Spreads_Dollar')
                spreads_percent.to_excel(writer, sheet_name='Spreads_Pct')
                spreads_percent_annual.to_excel(writer, sheet_name='Spreads_Annual')
                spreads_percent_annual_adjusted.to_excel(writer, sheet_name='Spreads_Annual_Adj')
                pd.DataFrame.from_dict(metadata, orient='index').to_excel(writer, sheet_name='Metadata')
                pd.Series(last_trade_dates).to_frame('Last_Trade_Date').to_excel(writer, sheet_name='LastTradeDates')
                interest_rates_df.to_excel(writer, sheet_name='Rates')
            
            print(f"Completed processing for {commodity}")
            return output_files
            
        finally:
            session.stop()
            
    except Exception as e:
        print(f"Error processing {commodity}: {str(e)}")
        return None

def create_observable_index(output_folder, results):
    """Create an index file for Observable with information about all commodities"""
    index = {
        'commodities': {},
        'date_range': {
            'start': str(datetime(FuturesParams.START_YEAR, 1, 1).date()),
            'end': str(datetime.now().date())
        },
        'last_updated': str(datetime.now())
    }
    
    for commodity, files in results.items():
        if files:
            with open(files['metadata'], 'r') as f:
                metadata = json.load(f)
            index['commodities'][commodity] = metadata
    
    index_file = os.path.join(output_folder, 'index.json')
    with open(index_file, 'w') as f:
        json.dump(index, f, indent=2)
    
    print(f"Created index file at {index_file}")
    return index_file

def main():
    print(f"""Initializing futures analysis with parameters:
    Minimum Volume: {FuturesParams.MIN_VOLUME}
    Minimum Days to Expiry: {FuturesParams.MIN_DAYS_TO_EXPIRY}
    Date Range: {FuturesParams.START_YEAR}-{FuturesParams.END_YEAR}
    Maximum Months Forward: {FuturesParams.MAX_MONTHS_FORWARD}
    Trading Days Per Year: {FuturesParams.TRADING_DAYS_PER_YEAR}
    Processing Commodities: {', '.join(FuturesParams.COMMODITIES)}
    """)
    
    start_date = datetime(FuturesParams.START_YEAR, 1, 1)
    end_date = datetime.now()
    
    # Process commodities sequentially
    results = {}
    
    for commodity in FuturesParams.COMMODITIES:
        try:
            print(f"\nProcessing {commodity}...")
            result = process_single_commodity(commodity, start_date, end_date)
            results[commodity] = result
            print(f"Completed processing {commodity}")
        except Exception as e:
            print(f"Error processing {commodity}: {str(e)}")
    
    # Create index file for Observable
    base_dir = os.getcwd()
    index_file = create_observable_index(base_dir, results)
    
    print("\nProcessing complete!")
    print(f"Data files saved in: {os.path.join(base_dir, 'data')}")
    print(f"Visualizations saved in: {os.path.join(base_dir, 'visualizations')}")
    print(f"Excel files saved in: {os.path.join(base_dir, 'excel')}")
    print(f"Index file created at: {index_file}")

if __name__ == "__main__":
    main()

Initializing futures analysis with parameters:
    Minimum Volume: 0
    Minimum Days to Expiry: 0
    Date Range: 1985-2026
    Maximum Months Forward: 13
    Trading Days Per Year: 251
    Processing Commodities: CLA, COA, XBA, HOA, NGA, FNA, HG, ALE, GC, SI
    

Processing CLA...

Starting processing for CLA
Generated 504 tickers for CLA

Processing metadata for CLA...
Processing metadata batch 1/3 for CLA
Processing metadata batch 2/3 for CLA
Processing metadata batch 3/3 for CLA
Collected metadata for 504 contracts

Processing price and volume data for CLA...
Processing batch 1/11 for CLA
Processing batch 2/11 for CLA
Processing batch 3/11 for CLA
Processing batch 4/11 for CLA
Processing batch 5/11 for CLA
Processing batch 6/11 for CLA
Processing batch 7/11 for CLA
Processing batch 8/11 for CLA
Processing batch 9/11 for CLA
Processing batch 10/11 for CLA
Processing batch 11/11 for CLA
Processing combined data with 2 columns
Processing raw data with 2 columns
Found 1 price columns

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Successfully fetched interest rate data

Organizing futures data for CLA...
Determined last trade dates for 1 contracts
Processing 1 dates...
Completed futures data organization. Processed 0 dates with valid data.
Exported dollar spreads to C:\Users\marti\OneDrive\Research\Python Code\Git Repository\futures-analysis\data\CLA\CLA_dollar_spreads.json
Exported percent spreads to C:\Users\marti\OneDrive\Research\Python Code\Git Repository\futures-analysis\data\CLA\CLA_percent_spreads.json
Exported annual spreads to C:\Users\marti\OneDrive\Research\Python Code\Git Repository\futures-analysis\data\CLA\CLA_annual_spreads.json
Exported adjusted spreads to C:\Users\marti\OneDrive\Research\Python Code\Git Repository\futures-analysis\data\CLA\CLA_adjusted_spreads.json
Exported metadata to C:\Users\marti\OneDrive\Research\Python Code\Git Repository\futures-analysis\data\CLA\CLA_metadata.json

Creating visualizations for CLA...
Plotting with 0 roll dates
Visualizations saved to C:\Users\marti\OneDr

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\marti\\OneDrive\\Research\\Python Code\\Git Repository\\futures-analysis\\data\\SI\\SI_metadata.json'